In [19]:
!pip install pypdf
!pip install faiss-cpu
!pip install tiktoken
!pip install qdrant-client


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Create vectorDb

In [20]:
!docker-compose up -d

 Container prototypenotebooks-qdrant-1  Running


In [21]:
! curl http://localhost:6333

{"title":"qdrant - vector search engine","version":"1.3.0"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    59  100    59    0     0  12322      0 --:--:-- --:--:-- --:--:-- 14750


In [22]:

from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter


### Same code as the gptLLM

In [ ]:
apiKey = ""

model_name = "Gpt"
llm = ChatOpenAI(openai_api_key=apiKey, model_name="gpt-3.5-turbo-16k")

### Read PDF

In [25]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

loader = PyPDFLoader("data/EStG.pdf")
pages = loader.load()
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = splitter.split_documents(pages)

In [26]:
print(docs)

[Document(page_content='Ein Service des Bundesministeriums der Justiz sowie des Bundesamts für\nJustiz ‒ www.gesetze-im-internet.de\n- Seite 1 von 280 -Einkommensteuergesetz  (EStG)\nEStG\nAusfertigungsdatum: 16.10.1934\nVollzitat:\n"Einkommensteuergesetz in der Fassung der Bekanntmachung vom 8. Oktober 2009 (BGBl. I S. 3366, 3862), das\nzuletzt durch Artikel 8 Absatz 3 des Gesetzes vom 20. Dezember 2022 (BGBl. I S. 2730) geändert worden ist"\nStand: Neugefasst durch Bek. v. 8.10.2009 I 3366, 3862;\n\xa0 zuletzt geändert durch Art. 8 Abs. 3 G v. 20.12.2022 I 2730\nFußnote\n(+++ Textnachweis Geltung ab: 30.12.1981 +++)\n(+++ Zur Anwendung vgl. § 4, § 4j, § 9, § 10, § 10a, § 13a, 13b, 19a, \n     § 20, § 22, § 32b, § 33, § 37, § 45e, §§ 52 ff. u.\n     § 92a +++)\n(+++ Zur Anwendung vgl. § 19 InvStG +++)\n(+++ Zur Anwendung d. § 34c vgl. § 26 KStG 1977 +++)\n(+++ Zur Anwendung vgl. §§ 6, 16, 19, 30, 31, 34, 38, 41, 42, 43, 45, 47, 49,  \n      50 u. 53 InvStG 2018 +++)\n\xa0\n \nÜberschr

### Here Vector database will be initialized

In [36]:
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings


# Loads the latest version
#prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

embeddings_model = OpenAIEmbeddings(openai_api_key=apiKey)
vectorDB = Qdrant.from_documents(docs, embeddings_model, collection_name="tax", host="localhost", port=6333)


In [37]:
template = """Use the following pieces of context to answer the question at the end. You can use your previous knowledge to answer the question.
Context: 
{context}
Question: {question}
Helpful Answer:"""
chat_prompt = PromptTemplate(input_variables=["context", "question"], template=template)
#QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)
qa = RetrievalQA.from_chain_type(llm,
                                chain_type='stuff',
                                retriever=vectorDB.as_retriever(search_kwargs={"k": 10}),
                                chain_type_kwargs={"prompt": chat_prompt})
 # k is the number of results to return


In [42]:
query="How much tax should I pay with earning 50k, single,  no kids?"#, k=3)
print(qa.run("What is the steuerklass 1?"))

'''question = "How much tax should I pay with no kids and wife earning 50k"
result = qa_chain({"question": query})
result["result"]'''

Steuerklasse 1 is a tax class in Germany that applies to single individuals who are either unlimited income taxpayers or limited income taxpayers.


'question = "How much tax should I pay with no kids and wife earning 50k"\nresult = qa_chain({"question": query})\nresult["result"]'

In [43]:
foundPages = vectorDB.similarity_search("What is the tax exemption regulation?", k=3)
print(foundPages[0].page_content)


Bekämpfung schädlicher Steuerpraktiken unter Berücksichtigung von Transparenz und Substanz“, OECD/
G20 Projekt Gewinnverkürzung und Gewinnverlagerung* , entspricht. 5Die Sätze 1 und 2 sind insoweit nicht
anzuwenden, als auf Grund der aus den Aufwendungen resultierenden Einnahmen ein Hinzurechnungsbetrag im
Sinne des § 10 Absatz 1 Satz 1 des Außensteuergesetzes anzusetzen ist.
(2) 1Eine niedrige Besteuerung im Sinne des Absatzes 1 liegt vor, wenn die von der Regelbesteuerung
abweichende Besteuerung der Einnahmen des Gläubigers oder des weiteren Gläubigers zu einer Belastung durch
Ertragsteuern von weniger als 25 Prozent führt; maßgeblich ist bei mehreren Gläubigern die niedrigste Belastung.
2Bei der Ermittlung, ob eine niedrige Besteuerung vorliegt, sind sämtliche Regelungen zu berücksichtigen,
die sich auf die Besteuerung der Einnahmen aus der Rechteüberlassung auswirken, insbesondere steuerliche
